In [5]:
import csv
import cv2
import numpy as np

lines = []

# reading the driving log file.
with open('/Users/vengal/Desktop/udacity self driving car nano degree/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)
        
# images contains input image figures and measurements has values of steering angle
images = []
measurements = []
# correction used for estimating angles from left and right imgae
correction = 0.2
for line in lines:
    source_path_center = line[0]
    source_path_left =  line[1]
    source_path_right = line[2]
    image_center = cv2.imread(source_path_center)
    image_left = cv2.imread(source_path_left)
    image_right = cv2.imread(source_path_right)
    images.append(image_center)
    images.append(image_left)
    images.append(image_right)
    measurements.append(float(line[3]))
    measurements.append(float(line[3])+correction)
    measurements.append(float(line[3])-correction)
    
agumented_images = []
agumented_measurements = []

# fliping the images and steering angle values and appending to data, so that more robust model
# can be trained with more data
for image, measurement in zip(images,measurements):
    agumented_images.append(image)
    agumented_images.append(cv2.flip(image,1))
    agumented_measurements.append(measurement)
    agumented_measurements.append(measurement*-1.0)
    
X_train = np.array(agumented_images)
Y_train = np.array(agumented_measurements)

from keras.models import Sequential
from keras.layers import Flatten,Dense,Lambda,Cropping2D,Dropout
from keras import layers

# below is the cnn used along with dropouts
model = Sequential()
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25), (0,0))))
model.add(layers.Conv2D(filters=24, kernel_size=(5, 5), activation='relu'))
model.add(layers.MaxPooling2D())
model.add(Dropout(0.5))
model.add(layers.Conv2D(filters=36, kernel_size=(5, 5), activation='relu'))
model.add(layers.MaxPooling2D())
model.add(Dropout(0.5))
model.add(layers.Conv2D(filters=48, kernel_size=(5, 5), activation='relu'))
model.add(layers.MaxPooling2D())
model.add(Dropout(0.5))
model.add(layers.Conv2D(filters=64, kernel_size=(3, 3)))
model.add(layers.Flatten())
model.add(layers.Dense(units=100))
model.add(Dropout(0.5))
model.add(layers.Dense(units=50))
model.add(Dropout(0.5))
model.add(layers.Dense(units=10))
model.add(Dropout(0.5))
model.add(layers.Dense(units=1))
#using adam optimizer
model.compile(loss='mse',optimizer='adam')
# traing and validating using 5% of data
model.fit(X_train,Y_train,validation_split=0.05,shuffle=True,epochs=3)
# saving the model
model.save('model.h5')

Train on 10624 samples, validate on 560 samples
Epoch 1/3
10624/10624 [==============================] - 521s 49ms/step - loss: 0.1165 - val_loss: 0.1343
Epoch 2/3
10624/10624 [==============================] - 470s 44ms/step - loss: 0.0700 - val_loss: 0.1420
Epoch 3/3
10624/10624 [==============================] - 426s 40ms/step - loss: 0.0672 - val_loss: 0.1373
